In [5]:
import pandas as pd
import MultiContagion as mc
import igraph
import random
import numpy as np
from matplotlib import pylab as plt
import scipy.stats
import copy
from scipy.sparse import linalg as LA
#import powerlaw
%matplotlib inline

In [6]:
year = 2015

In [7]:
#import the aggregate adjacency matrix
aggregate_am = np.genfromtxt ('csv_files/AM4_all_nodes_aggregateNorm'+str(year)+'.csv', delimiter=",")
df_names = pd.read_csv('csv_files/all_country_name4.csv', header=None)
names = list(df_names[0])
Aggregate_g = igraph.Graph.Weighted_Adjacency(list(np.transpose(aggregate_am)))
Aggregate_g.vs["name"] = copy.deepcopy(names)

In [8]:
#Import the adjacency matrix and make graphs of all the layers.
cdis_equity_am = np.genfromtxt ('csv_files/AM4_all_nodesNormCDIS-equity'+str(year)+'.csv', delimiter=",")
cdis_equity_g = igraph.Graph.Weighted_Adjacency(list(cdis_equity_am))
cdis_equity_g.vs["name"] = copy.deepcopy(names)

cdis_debt_am = np.genfromtxt ('csv_files/AM4_all_nodesNormCDIS-debt'+str(year)+'.csv', delimiter=",")
cdis_debt_g = igraph.Graph.Weighted_Adjacency(list(cdis_debt_am))
cdis_debt_g.vs["name"] = copy.deepcopy(names)

cpis_equity_am = np.genfromtxt ('csv_files/AM4_all_nodesNormCPIS-equity'+str(year)+'.csv', delimiter=",")
cpis_equity_g = igraph.Graph.Weighted_Adjacency(list(cpis_equity_am))
cpis_equity_g.vs["name"] = copy.deepcopy(names)

cpis_debt_am = np.genfromtxt ('csv_files/AM4_all_nodesNormCPIS-debt'+str(year)+'.csv', delimiter=",")
cpis_debt_g = igraph.Graph.Weighted_Adjacency(list(cpis_debt_am))
cpis_debt_g.vs["name"] = copy.deepcopy(names)

bis_am = np.genfromtxt ('csv_files/AM4_all_nodesNormBIS'+str(year)+'.csv', delimiter=",")
bis_g = igraph.Graph.Weighted_Adjacency(list(bis_am))
bis_g.vs["name"] = copy.deepcopy(names)

In [9]:
G_list = [cdis_equity_g, cdis_debt_g, cpis_equity_g, cpis_debt_g, bis_g ]

In [10]:
AM_list = [np.transpose(cdis_equity_am), np.transpose(cdis_debt_am), np.transpose(cpis_equity_am), np.transpose(cpis_debt_am), np.transpose(bis_am) ]

In [11]:
G_list2 = [cdis_equity_g,cpis_equity_g, cpis_debt_g, bis_g ]
AM_list2 = [cdis_equity_am, cpis_equity_am, cpis_debt_am, bis_am ]

In [12]:
In_list = [ np.array(g.strength(mode ="IN", weights= g.es["weight"])) for g in G_list]
Out_list = [ np.array(g.strength(mode ="OUT", weights= g.es["weight"])) for g in G_list]

In [13]:
def make_supra_coup(list_adj, list_vector):
    "Takes a list of adjacency matrix and list of vectors to use as coupling for layers"
    n_layers = len(list_adj)
    dim = list_adj[0].shape[0]
    sup_list = []
    for i in range(n_layers):
        row = []
        for j in range(n_layers):
            if i == j:
                row.append(list_adj[i])
            else:
                row.append(np.diag(list_vector[j]))
        sup_list.append(row)
    return np.bmat(sup_list)

# For real

In [14]:
supra_adj = make_supra_coup(AM_list, In_list)
G_supra = igraph.Graph.Weighted_Adjacency(supra_adj.tolist())
complete_multi_pr = G_supra.personalized_pagerank(weights = G_supra.es["weight"])

In [15]:
G_supra = igraph.Graph.Weighted_Adjacency(supra_adj.tolist())

In [16]:
G_supra = igraph.Graph.Weighted_Adjacency(supra_adj.tolist())

In [17]:
G_supra.is_weighted(), G_supra.is_directed()

(True, True)

In [18]:
def save_centrality(y, namefile, namecent , country, cent):
    f = open("centralities/"+namefile+str(y) +".csv", "w")
    f.write("Country,"+namecent+"\n")
    n = len(country)
    for i in range(n):
        f.write(str(country[i]) + "," + str(cent[i])+ "\n")
    f.close()

## PageRank

In [19]:
pr_agg = Aggregate_g.personalized_pagerank(weights = Aggregate_g.es["weight"], damping = 0.85)
node_names = np.array(copy.deepcopy(names))
inds = np.array(pr_agg).argsort()[::-1][:]
sort_names = node_names[inds]
pr_agg_ord = np.array(pr_agg)[inds]

In [20]:
sort_names[:30]

array(['United States', 'Netherlands', 'United Kingdom', 'Germany',
       'France', 'Luxembourg', 'Switzerland', 'Ireland',
       'China  P.R.: Hong Kong', 'Belgium', 'Mauritius', 'Italy', 'Spain',
       'Brazil', 'Russian Federation', 'Canada', 'China  P.R.: Mainland',
       'Australia', 'Mexico', 'Cayman Islands', 'Virgin Islands  British',
       'Singapore', 'Sweden', 'Bermuda', 'India', 'Japan', 'Cyprus',
       'Austria', 'Norway', 'Poland'], 
      dtype='|S36')

In [21]:
save_centrality(year, "Agg_PageRank", "PageRank", sort_names, pr_agg_ord)

In [22]:
def get_pagerank_list(G, n_layers = 5, names= names, r = 0.85):
    pr = G.personalized_pagerank(weights = G.es["weight"], damping = r)
    multi_pr = []
    n_nodes = int((len(G.degree()))/5)
    multi_pr = []
    for i in range(n_nodes):
        cent = 0
        for k in range(n_layers):
            cent += pr[i + n_nodes*k] 
        multi_pr.append(cent)
    node_names = np.array(copy.deepcopy(names))
    inds = np.array(multi_pr).argsort()[::-1][:]
    sort_names_multi_pr = node_names[inds]
    multi_sort_pr = np.array(multi_pr)[inds]
    return sort_names_multi_pr, multi_sort_pr

In [23]:
n_pr, num_pr = get_pagerank_list(G_supra, r = 0.85)

In [24]:
n_pr, num_pr = get_pagerank_list(G_supra, r = 0.85)
n_pr[:30]

array(['United States', 'United Kingdom', 'Netherlands', 'Germany',
       'France', 'Luxembourg', 'Switzerland', 'Ireland',
       'China  P.R.: Hong Kong', 'Mauritius', 'Belgium', 'Italy', 'Spain',
       'Brazil', 'Canada', 'Russian Federation', 'Australia',
       'China  P.R.: Mainland', 'Mexico', 'Singapore', 'Cayman Islands',
       'Virgin Islands  British', 'Sweden', 'Japan', 'Bermuda', 'India',
       'Cyprus', 'Austria', 'Norway', 'South Africa'], 
      dtype='|S36')

In [25]:
def make_supra_coup2(list_adj, list_vector):
    "Takes a list of adjacency matrix and list of vectors to use as coupling for layers"
    n_layers = len(list_adj)
    dim = list_adj[0].shape[0]
    sup_list = []
    for i in range(n_layers):
        row = []
        for j in range(n_layers):
            if i == j:
                row.append(list_adj[i])
            else:
                row.append(np.diag(list_vector[i]))
        sup_list.append(row)
    return np.bmat(sup_list)

In [26]:
supra_adj2 = make_supra_coup2(AM_list, In_list)
G_supra2 = igraph.Graph.Weighted_Adjacency(supra_adj2.tolist())

In [27]:
n_pr2, num_pr2 = get_pagerank_list(G_supra2)

In [28]:
n_pr2;

In [29]:
n_pr2, num_pr2 = get_pagerank_list(G_supra2, r = 0.85)
n_pr2[:30]

array(['United States', 'United Kingdom', 'Netherlands', 'Luxembourg',
       'France', 'Germany', 'China  P.R.: Mainland',
       'China  P.R.: Hong Kong', 'Cayman Islands', 'Switzerland', 'Japan',
       'Ireland', 'Italy', 'Canada', 'Mauritius', 'Australia', 'Spain',
       'Belgium', 'India', 'Brazil', 'Russian Federation', 'Singapore',
       'Mexico', 'Austria', 'Sweden', 'Korea  Republic of', 'Thailand',
       'Panama', 'South Africa', 'Bermuda'], 
      dtype='|S36')

In [30]:
save_centrality(year, "MultCoupCashFlow_PageRank", "PageRank centrality", n_pr, num_pr)

In [31]:
save_centrality(year, "MultCoupRiskFlow_PageRank", "PageRank centrality", n_pr2, num_pr2)

## Eigenvector

In [32]:
def get_eigenvector_list(G, n_layers = 5, names= names):
    pr = G.eigenvector_centrality(weights = G.es["weight"])
    multi_pr = []
    n_nodes = int((len(G.degree()))/5)
    multi_pr = []
    for i in range(n_nodes):
        cent = 0
        for k in range(n_layers):
            cent += pr[i + n_nodes*k] 
        multi_pr.append(cent)
    node_names = np.array(copy.deepcopy(names))
    inds = np.array(multi_pr).argsort()[::-1][:]
    sort_names_multi_pr = node_names[inds]
    multi_sort_pr = np.array(multi_pr)[inds]
    return sort_names_multi_pr, multi_sort_pr

In [33]:
n_ei, num_ei = get_eigenvector_list(G_supra)
n_ei2, num_ei2 = get_eigenvector_list(G_supra2)

/home/user/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: Weighted directed graph in eigenvector centrality at centrality.c:362
  from ipykernel import kernelapp as app


In [34]:
n_ei2;

In [35]:
save_centrality(year, "MultCoupCashFlow_Eigenvector", "Eigenvector centrality", n_ei, num_ei)
save_centrality(year, "MultCoupRiskFlow_Eigenvector", "Eigenvector centrality", n_ei2, num_ei2)

## Hubs and Auth

In [36]:
def G_list_hub(G_adj_list):
    G_hub = []
    for g in G_adj_list:
        new = np.dot(g, g.transpose())
        G_hub.append(new)   
    return G_hub

In [37]:
def G_list_auth(G_adj_list):
    G_hub = []
    for g in G_adj_list:
        new = np.dot( g.transpose(), g)
        G_hub.append(new)   
    return G_hub

In [38]:
def hub_auth_diag(Coup_list):
    coup = []
    for c in Coup_list:
        coup.append(c*c)
    return coup

In [39]:
G_supra_hub = make_supra_coup(G_list_hub(AM_list), hub_auth_diag(In_list))
G_supra_auth = make_supra_coup(G_list_auth(AM_list), hub_auth_diag(In_list))

In [40]:
G_supra_hub[0].shape

(1, 1065)

In [41]:
def get_hub_list(AM, n_layers = 5, names= names):
    hub_eig = LA.eigs(AM, k=1, which='LM')[1]
    multi_pr = []
    #print hub_eig[0]
    n_nodes = int((AM[0].shape[1])/5.)
    multi_pr = []
    for i in range(n_nodes):
        cent = 0
        for k in range(n_layers):
            
            cent += abs(hub_eig[i + n_nodes*k])
        
        multi_pr.append(cent[0])
    
    node_names = np.array(copy.deepcopy(names))
    inds = np.array(multi_pr).argsort()[::-1][:]
    sort_names_multi_pr = node_names[inds]
    multi_sort_pr = np.array(multi_pr)[inds]
    return sort_names_multi_pr, multi_sort_pr

In [42]:
n_hub, num_hub = get_hub_list(G_supra_hub)
n_auth, num_auth = get_hub_list(G_supra_auth)

In [43]:
n_auth, num_auth = get_hub_list(G_supra_auth)

In [44]:
n_auth[:20]

array(['Netherlands', 'United States', 'Luxembourg', 'United Kingdom',
       'Germany', 'Ireland', 'Belgium', 'Switzerland', 'France', 'Spain',
       'Brazil', 'Bermuda', 'Italy', 'Canada', 'Australia',
       'China  P.R.: Hong Kong', 'Russian Federation', 'Sweden',
       'Cayman Islands', 'Japan'], 
      dtype='|S36')

In [45]:
n_hub[:20]

array(['Netherlands', 'United States', 'Luxembourg', 'United Kingdom',
       'Belgium', 'Ireland', 'Germany', 'Switzerland', 'France', 'Bermuda',
       'China  P.R.: Hong Kong', 'Cayman Islands', 'Canada', 'Sweden',
       'Italy', 'Japan', 'Spain', 'Mexico', 'Singapore', 'Jersey'], 
      dtype='|S36')

In [46]:
save_centrality(year, "MultCoupCashFlow_Hub", "Hub Score", n_hub, num_hub)
save_centrality(year, "MultCoupCashFlow_Auth", "Authority Score", n_auth, num_auth)

In [47]:
n_auth[:20]

array(['Netherlands', 'United States', 'Luxembourg', 'United Kingdom',
       'Germany', 'Ireland', 'Belgium', 'Switzerland', 'France', 'Spain',
       'Brazil', 'Bermuda', 'Italy', 'Canada', 'Australia',
       'China  P.R.: Hong Kong', 'Russian Federation', 'Sweden',
       'Cayman Islands', 'Japan'], 
      dtype='|S36')

In [48]:
n_hub[:20]

array(['Netherlands', 'United States', 'Luxembourg', 'United Kingdom',
       'Belgium', 'Ireland', 'Germany', 'Switzerland', 'France', 'Bermuda',
       'China  P.R.: Hong Kong', 'Cayman Islands', 'Canada', 'Sweden',
       'Italy', 'Japan', 'Spain', 'Mexico', 'Singapore', 'Jersey'], 
      dtype='|S36')

### other coup

In [49]:
G_supra_hub2 = make_supra_coup2(G_list_hub(AM_list), hub_auth_diag(In_list))
G_supra_auth2 = make_supra_coup2(G_list_auth(AM_list), hub_auth_diag(In_list))

In [50]:
n_hub2, num_hub2 = get_hub_list(G_supra_hub2)
n_auth2, num_auth2 = get_hub_list(G_supra_auth2)

In [51]:
#n_hub2, num_hub2 = get_eigenvector_list(igraph.Graph.Weighted_Adjacency(G_supra_hub2.tolist()))
#n_auth2, num_auth2 = get_eigenvector_list(igraph.Graph.Weighted_Adjacency(G_supra_auth2.tolist()))

In [52]:
n_auth2[:30]

array(['United States', 'Germany', 'United Kingdom', 'Netherlands',
       'Ireland', 'Luxembourg', 'Switzerland', 'Spain', 'France',
       'Belgium', 'Brazil', 'Italy', 'Australia', 'Canada', 'Bermuda',
       'Russian Federation', 'Sweden', 'Japan', 'Cayman Islands', 'Mexico',
       'Norway', 'China  P.R.: Mainland', 'Singapore',
       'China  P.R.: Hong Kong', 'United Arab Emirates', 'Hungary',
       'Mauritius', 'Poland', 'Puerto Rico', 'Austria'], 
      dtype='|S36')

In [53]:
n_hub2[:30]

array(['Netherlands', 'Luxembourg', 'United States', 'United Kingdom',
       'Belgium', 'Ireland', 'Switzerland', 'France', 'Germany', 'Bermuda',
       'Cayman Islands', 'Canada', 'Sweden', 'Italy', 'Japan', 'Mexico',
       'Spain', 'China  P.R.: Hong Kong', 'Singapore', 'Jersey', 'Hungary',
       'China  P.R.: Mainland', 'Australia', 'United Arab Emirates',
       'Denmark', 'Russian Federation', 'Austria', 'Isle of Man', 'Norway',
       'Finland'], 
      dtype='|S36')

In [54]:
save_centrality(year, "MultCoupRiskFlow_Hub", "Hub Score", n_hub2, num_hub2)
save_centrality(year, "MultCoupRiskFlow_Auth", "Authority Score", n_auth2, num_auth2)

In [55]:
num_hub2 [:20]

array([ 0.64354552,  0.41965027,  0.37155333,  0.22015778,  0.21670951,
        0.19763922,  0.19235927,  0.17597517,  0.16413228,  0.118482  ,
        0.07241298,  0.06994872,  0.04819354,  0.04729882,  0.04482315,
        0.04187269,  0.04084805,  0.03810544,  0.03725383,  0.0354914 ])